# Content
     
**Data Preprocessing**  
   - Settings
   - Loading Data
   - Glossary
   - Dealing with missing Values
   - Fixing Data Types
   - Dealing Bad Values
   - Feature Engieenier

# Data Preprossesing

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime, date, timedelta
from tabulate import tabulate
from IPython.display import HTML
import dataframe_image as di
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def menu_settings():
    
    display(HTML("""
            <style>

            h1 {
            background-color: SteelBlue;
            color: white;
            padding: 15px 15px;
            text-align: center;
            font-family: Arial, Helvetica, sans-serif;
            border-radius:10px 10px;
            }

            h2 {
            background-color: CadetBlue;
            color: white;
            padding: 10px 10px;
            text-align: center;
            font-family: Arial, Helvetica, sans-serif
            border-radius:10px 10px;
            }

            </style>            
    """))

In [3]:
menu_settings()

## Loading Data

In [4]:
data_raw = pd.read_csv('../data/ecommerce.csv', encoding='iso-8859-1')
data = data_raw.copy()

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
 8   Unnamed: 8   0 non-null       float64
dtypes: float64(3), int64(1), object(5)
memory usage: 37.2+ MB


## Glossary

In [6]:
glossary = [['Columns', 'Meaning'],
            ['InvoiceNo', 'Unique Identifier of each transaction'],
            ['StockCode', 'Internal item code'],
            ['Description', 'Item description/resume'],
            ['Quantity', 'Quantity of each item per transaction'],
            ['InvoiceDate', 'The day of transaction'],
            ['UnitPrice', 'Product price per unit'],
            ['CustomerID', 'Unique Identifier of Customer'],
            ['Country', 'Customer\'s country of residence']
           ]
#print(tabulate(glossary, headers='firstrow', stralign='left', tablefmt='simple'))

## Dealing with missing values

In [7]:
data.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
Unnamed: 8     541909
dtype: int64

In [8]:
data = data.drop('Unnamed: 8', axis=1)
data = data.dropna(subset=['CustomerID'])#'Description',

As the purpose of this project is to group customers, then it makes no sense to classify unidentified customers. To simplify the study, we will initially ignore unidentified customers, which are those who purchased but we do not know who they are because at the time of purchase, he or she was not a registered user or was not informed at the time of purchase.

## Fixing Data Types

In [9]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], format='%d-%b-%y') #format='%m/%d/%Y %H:%M'
data['CustomerID'] = data['CustomerID'].astype(int)

data['Total'] = data['Quantity'] * data['UnitPrice']

In [10]:
#data['date'] = pd.to_datetime(data['date'], format="%Y-%m-%d")
# data['year'] = pd.DatetimeIndex(data['InvoiceDate']).year
# data['month'] = pd.DatetimeIndex(data['InvoiceDate']).month
# data['week_of_year'] = data['InvoiceDate'].dt.isocalendar().week
# data['day'] = pd.DatetimeIndex(data['InvoiceDate']).day

In [11]:
data.InvoiceDate.min(), data.InvoiceDate.max()

(Timestamp('2016-11-29 00:00:00'), Timestamp('2017-12-07 00:00:00'))

## Dealing Bad Values

In [12]:
sum_transactions_per_client=data[['CustomerID','Total','Quantity']].groupby('CustomerID').agg({'Total':np.sum,
                                                                 'Quantity':np.sum,                               
                                                                 #'CustomerID':np.unique                                                                                    
                                                                 }).reset_index()

In [13]:
#Customers who do not have a positive purchase balance or who owe the company (due to the temporal cut of the database) will be excluded

bad_clients = sum_transactions_per_client.loc[(sum_transactions_per_client['Total'] <= 0.5) | (sum_transactions_per_client['Quantity'] <= 1)]

In [14]:
list_bad_clients=bad_clients['CustomerID'].tolist()
data = data[~data['CustomerID'].isin(list_bad_clients)]

In [15]:
data = data.loc[~(data['UnitPrice'] < 0.04)]

In [16]:
data[data.StockCode.str.contains("^[a-zA-Z]")].StockCode.value_counts()

POST            1196
M                434
C2               134
D                 75
DOT               16
CRUK              16
BANK CHARGES      10
Name: StockCode, dtype: int64

In [17]:
data[data.StockCode.str.contains("^[a-zA-Z]")].Description.value_counts()

POSTAGE            1196
Manual              434
CARRIAGE            134
Discount             75
DOTCOM POSTAGE       16
CRUK Commission      16
Bank Charges         10
Name: Description, dtype: int64

In [18]:
list_letter_stock=data[data.StockCode.str.contains("^[a-zA-Z]")].StockCode.value_counts().index.tolist()

In [19]:
data = data.loc[~data['StockCode'].isin(list_letter_stock)]

In [20]:
data.groupby("StockCode")["Description"].nunique()[data.groupby("StockCode")["Description"].nunique() != 1]

StockCode
16156L    2
17107D    3
20622     2
20725     2
20914     2
         ..
85184C    2
85185B    2
90014A    2
90014B    2
90014C    2
Name: Description, Length: 213, dtype: int64

Absurd purchases followed by cancellations, purchase values close to or below zero will be considered as bad input values and thus will be deleted. They can even be useful in the EDA stage to generate insights, but for the machine learning model they significantly interfere with performance.

As this database is a temporal cut of the company's sales, we will find purchase cancellations but we will not find the purchase related to this cancellation, this is a big problem. One of the ways to solve this is to identify the cancellations one by one and delete this line, another way is to delete the customers that on average the company owes them. I preferred to choose the second way because it is simpler to perform, later in the code this will be done.

## Feature Engeenier

In [21]:
transactions = data.copy()

In [33]:
#Group InvoiceNumber, it contains sales and cancelations

transactions=data.groupby('InvoiceNo').agg( CustomerID = ('CustomerID', np.unique),
                                            InvoiceDate = ('InvoiceDate', np.unique),
                                            Total = ('Total', 'sum'),
                                            UniqueProducts = ('StockCode', 'nunique'), 
                                            Items = ('Quantity', 'sum'),
                                            Country = ('Country', np.unique),
                                            ProductsCode = ('StockCode', np.unique)).reset_index()

transactions['AvarageTicket']= round(transactions['Total']/transactions['UniqueProducts'],2) 
#len(transactions)

In [34]:
#transactions.InvoiceNo.str.contains("C").value_counts()

In [35]:
transactions

,InvoiceNo,CustomerID,InvoiceDate,Total,UniqueProducts,Items,Country,ProductsCode,AvarageTicket
0,536365,17850,2016-11-29,139.12,7,40,United Kingdom,"[21730, 22752, 71053, 84029E, 84029G, 84406B, ...",19.87
1,536366,17850,2016-11-29,22.20,2,12,United Kingdom,"[22632, 22633]",11.10
2,536367,13047,2016-11-29,278.73,12,83,United Kingdom,"[21754, 21755, 21777, 22310, 22622, 22623, 227...",23.23
3,536368,13047,2016-11-29,70.05,4,15,United Kingdom,"[22912, 22913, 22914, 22960]",17.51
4,536369,13047,2016-11-29,17.85,1,3,United Kingdom,21756,17.85
...,...,...,...,...,...,...,...,...,...
21700,C581470,17924,2017-12-06,-8.32,1,-4,United Kingdom,23084,-8.32
21701,C581484,16446,2017-12-07,-168469.60,1,-80995,United Kingdom,23843,-168469.60
21702,C581490,14397,2017-12-07,-32.53,2,-23,United Kingdom,"[22178, 23144]",-16.26
21703,C581568,15311,2017-12-07,-54.75,1,-5,United Kingdom,21258,-54.75


In [36]:
last_day = data.InvoiceDate.max() + dt.timedelta(days = 1)

transactions_per_customer = transactions.groupby('CustomerID').agg(
                                                      GrossRevenue = ('Total', np.sum),                                           
                                                      Recency = ('InvoiceDate', lambda x: ((last_day - x.max()).days)),             
                                                      Frequency = ('InvoiceNo', 'count'),             
                                                      Products = ('UniqueProducts', 'sum'), 
                                                      Items = ('Items', 'sum'),  
                                                      Country = ('Country', np.unique),
                                                      AvarageTicket = ('AvarageTicket', 'sum'))            
                                                      #Products = ('StockCode', np.unique),
                                                                   
#transactions_per_customer['AvarageTicket']= round(transactions_per_customer['GrossRevenue'] / transactions_per_customer['Products'],2)